In [ ]:
#Necessary libraries
import requests as rq
from bs4 import BeautifulSoup as bs
import re #regular expressions
from datetime import date
import datetime
import pandas as pd
import calendar
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Set up the DF for the scraped data
game_cols = ['Day_Name'
             , 'Month_Name'
             , 'Day_Of_Month'
             , 'Year'
             , 'Week'
             , 'Home_Team'
             , 'Away_Team'
             , 'Home_Score'
             , 'Away_Score'
             , 'Home_wins'
             , 'Home_losses'
             , 'Home_draws'
             , 'Away_wins'
             , 'Away_losses'
             , 'Away_draws'
             , 'Winner'
            ]
game_data = pd.DataFrame(columns = game_cols)

In [ ]:
#initialize the overall counter number
m = 0
#initialize a way to break out of the outer loop. This is a goofy-looking kludge, but it works 
break_flag = 0

#initialize a blank dictionary for the team records
#I can get this out of the page eventually, but right now I'll keep track myself
records = {}

for i in range(18): #18
    #this is a kludge to break out of the outer for loop if there is no game
    if break_flag != 0:
        break
        
    #initialize the counter of games in the week
    count_games = 0
    
    week = i+1
    #set the url
    test_url = "https://www.espn.com/nfl/schedule/_/week/" + str(week) + "/year/2023/seasontype/2"
    
    #set the header so that espn doesn't reject me directly
    header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"} #have to do this or else ESPN will block me for being a python script and not a user.
    request = rq.get(test_url, headers = header) #assign my request to a variable

    #get the html of the page
    html_text = bs(request.text, 'html') #See the HTML code of the whole page
    
    #get the away and home names
        #have to do it this way because searching for 'Table_Team' will also grab 'Table_Team away'
    all_names = html_text.find_all('span', class_ = 'Table__Team')
    away_names = all_names[::2]
    home_names = all_names[1::2]
    
    #every day's group of games is in a separate group.
    game_groups = html_text.find_all('tbody')
    
    for j in range(len(game_groups)):
        
        scores = game_groups[j].find_all('a', class_ = 'AnchorLink', tabindex = '0', href=re.compile('/nfl/game\?'))
        
        for k in range(len(scores)):
            #print('week = ', week, 'scores text = ', scores[k].text)
            if (scores[k].text != 'Canceled' 
                and re.match('.*(?:AM|PM).*', scores[k].text) is None 
                and scores[k].text != 'TBD'):
                
                #don't run through this section if AM, PM, or TBD
                
                #get away team abbreviation
                away_team = re.split('/', away_names[count_games].find_all('a', href=re.compile('/nfl/team/_/name/'))[1]['href'])[5].upper()
                
                #get home team abbreviation
                home_team = re.split('/', home_names[count_games].find_all('a', href=re.compile('/nfl/team/_/name/'))[1]['href'])[5].upper()
                
                #site shows results as 'winner, loser' rather than 'home, away'
                
                try:
                    print('scores text = ', scores[k].text)
                    win_team, win_score, throw_away, lose_team, lose_score = re.split('[ |, ]', scores[k].text)[:5]
                    print('win team = ', win_team)
                    print('win score = ', win_score)
                    print('lose team = ', lose_team)
                    print('lose score = ', lose_score)
                    print('home team = ', home_team)
                    print('away team = ', away_team)
                #if error then break
                except:
                    #set the flag to break out of the outer for loop if there is no game
                    break_flag = 1
                    print('break')
                    break
                
                if win_team == home_team:
                    home_score = int(win_score)
                    away_score = int(lose_score)
                if win_team == away_team:
                    home_score = int(lose_score)
                    away_score = int(win_score)
                    
                #main -> div (Table__title) -> dates
                dates = html_text.find_all('div', class_ = 'Table__Title') #on the base html code, returns a list of the full tags
                day_name, month_name, day_of_month, year = re.split(' |, ', dates[j].text)[:4] #split the text of the date by commas or spaces
                #print(day_name, month_name, day_of_month, year)

                #Team records are actually in the html.
                #They just aren't displayed
                    #<div class="ScoreCell__Score ScoreCell__Score--record n9 tar">
        
                #if a team does not have an entry in the records dictionary then initialize one
                if home_team not in records:
                    records[home_team] = [0,0,0]
                if away_team not in records:
                    records[away_team] = [0,0,0]
                    
                #assign wins and losses based on scores
                if home_score > away_score:
                    records[home_team][0] += 1
                    records[away_team][1] += 1
                    winner = 'H'
                elif away_score > home_score:
                    records[away_team][0] += 1
                    records[home_team][1] += 1
                    winner = 'A'
                else:
                    records[away_team][2] += 1
                    records[home_team][2] += 1
                    winner = 'D'
                                     
                #append to the DF of the games
                game_data.loc[m] = [ 
                    day_name
                    , month_name
                    , day_of_month
                    , year
                    , week
                    , home_team
                    , away_team
                    , home_score
                    , away_score
                    , records[home_team][0]
                    , records[home_team][1]
                    , records[home_team][2]
                    , records[away_team][0]
                    , records[away_team][1]
                    , records[away_team][2]
                    , winner
                    ]
                
                #count the games so things line up right
                count_games += 1
                #increment the row counter    
                m += 1

In [ ]:
game_data.tail(16)

In [ ]:
game_data_played = game_data.drop(['Home_Score', 'Away_Score'], axis = 1)
game_data_played1 = game_data_played.copy(deep=True) #deep copy this before I change it

In [ ]:
factor = pd.factorize(game_data_played['Winner'])
game_data_played['Winner'] = factor[0]
definitions = factor[1]
print(game_data_played['Winner'].head(16))
print(definitions)
print(game_data_played['Winner'].value_counts())

In [ ]:
game_data_played.tail(16)

In [ ]:
#processing pipeline needs to include
    #OHE of all categorical fields
        #Day_Name
        #Month_Name
        #Home_Team
        #Away_Team
#processing pipeline DOES NOT need to include factorizing 'Winner', 
    #though it may or may not be helpful to de-factorize at the end

In [ ]:
game_data_played = pd.get_dummies(game_data_played, columns=['Day_Name', 'Month_Name', 'Home_Team', 'Away_Team'], drop_first=False)
print(game_data_played)

In [ ]:
print(game_data_train.shape)

In [ ]:
#print(records['LAR'])
#print(records['BUF'])

In [ ]:
game_data_train = game_data_played.drop(['Winner'], axis = 1)
game_data_values = game_data_played['Winner']

In [ ]:
#manually define next game(s)
game_data_unplayed = [['Saturday', 'December', '23', '2023', 16, 'PIT', 'CIN', 7, 7, 0, 8, 6, 0], ['Saturday', 'December', '23', '2023', 16, 'LAR', 'BUF', 8, 7, 0, 8, 6, 0]]

new_data = pd.get_dummies(pd.DataFrame(game_data_unplayed, columns = game_data_train.columns), columns = game_data_train.columns)
print(new_data.shape)
new_data

#gives me 2 rows, 16 columns.
#need 225